In [465]:
import pandas as pd
import numpy as np
import re, string
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


# Part 1: Multinomial Naive Bayes

In [206]:
#a list of text from all authors
folder = []
for i in range(0, 23):
    for j in range(0,15):
        path = "articles/train/author_"+str(i+1)+ "/" + str(j+1)+ ".txt"
        with open(path,'r', encoding = "utf8") as file:
            p = file.read()
            p = p.lower()
            folder.append(p)

In [207]:
#dictionary for text of each author
authors = ["author_1","author_2","author_3","author_4","author_5","author_6","author_7","author_8"
             ,"author_9","author_10","author_11","author_12","author_13","author_14","author_15","author_16","author_17"
             ,"author_18","author_19","author_20","author_21","author_22","author_23"]

# dictionary = {author: None for author in authors}
dictionary = dict([(author, []) for author in authors])

for i in range(0, 23):
    author = "author_"+str(i+1)
    for j in range(0,15):
        path = "articles/train/author_"+str(i+1)+ "/" + str(j+1)+ ".txt"
        with open(path,'r', encoding = "utf8") as file:
            p = file.read()
            p = p.lower()
            dictionary[author].append(p)



In [309]:
#dictionary for text of each author
authors_test = ["author_1","author_2","author_3","author_4","author_5","author_6","author_7","author_8"
             ,"author_9","author_10","author_11","author_12","author_13","author_14","author_15","author_16","author_17"
             ,"author_18","author_19","author_20","author_21","author_22","author_23"]

# dictionary = {author: None for author in authors}
dictionary_test = dict([(author, []) for author in authors_test])

for i in range(1, 24):
    author = "author_"+str(i)
    for j in range(16,21):
        path = "articles/test/author_"+str(i)+ "/" + str(j)+ ".txt"
        with open(path,'r', encoding = "utf8") as file:
            p = file.read()
            p = p.lower()
            dictionary_test[author].append(p)

In [310]:
# dictionary_test

{'author_1': ['trying the blogthis ! link in the tool bar . seems to work . been having trouble posting . as noted by the double blog on my weekend . also took down mannequin pic from profile . damn . ',
  "cleaned . rearranged . made a mess . cleaned . the vicious cycle of trying to make my place presentable ... or at the very least less cluttered . now most of my plastic army is on both my computer screens . their little eyes staring blankly out at the opposite wall behind me . the monsters are arranged in a cube/crate thing looking fierce and dangerous . and whole bunch of comics and other misc . crap is on the dining room table . my mannequin now has a somewhat decent wardrobe after i went through my closet . you can see her all decked out in my profile picture . found more shirts that i forgot i had , lying on a shelf . hung them up so i remember that i have them and can incorporate them into my daily wear . room 's nowhere near looking orderly though . tthhhrrppp ",
  "happy 4th 

In [223]:
# (dictionary['author_1'])

["have an old clamshell ibook that i found on consignment . works pretty well . it was cheaper than what was being sold on ebay and those were slower models . added more memory so could upgrade os . now runs panther decently . added a bunch of games that i just trashed tonight to make more room on the hard drive ( only has 10 gigs ) . also am trying to find replacement products for the microsoft products on it . took off office and installed openoffice . use netscape and safari instead of ie . not exactly su re why i 'm doing it though . it risks screwing up my computer , but i find it a fun experiment . i am such a mac geek.m ",
 "not sure if i mentioned in a previous post , but i hate the heat . i am not a sun bunny . do n't like outdoor activities . no beach , no camping , no hiking . do n't really like to sweat . which kinda sucks since i live in hawaii . i really should be used to it , having been born and raised here , but i really do n't like the sun . i avoid the beach like the

In [208]:
# df = pd.DataFrame.from_dict(dictionary)
# #combine all texts together
# df['author_1'].str.cat(sep=', ')
documents = []
for i in folder:
    text = i.lower()
    processed_text = re.sub(r'[^\w\s]','',text)
    documents.append(processed_text)
    


In [325]:
########## TRAIN DATA #############
count_vect = CountVectorizer(stop_words = 'english',max_features = 15000, ngram_range = (1,2))
#create a vocabulary set - a matrix of vocabulary
count_vectorizer = count_vect.fit_transform(documents)
X = count_vectorizer.toarray()
y = []
for i in range(1,24): #authors
    for j in range(0,15):
        y.append(i)

#Multinomial NB
mnb_clf = MultinomialNB(alpha = 0.001)
mnb_clf.fit(X,y)
mnb_accuracy = cross_val_score(mnb_clf, X,y, cv = 10)
np.mean(mnb_accuracy)

0.7456521739130435

In [210]:
########## PROCESSING TRAIN DATA #############
import string
test_folder = []
for i in range(1, 24): #the authors
    for j in range(16,21): #the articles
        path = "articles/test/author_"+str(i)+ "/" + str(j)+ ".txt"
        with open(path,'r', encoding = "utf8") as file:
            p = file.read()
            p = p.lower()
            test_folder.append(p)
         
test_documents = []
for i in test_folder:
    test_text = i.lower()
    test_processed_text = re.sub(r'[^\w\s]','',test_text)
    test_documents.append(test_processed_text)

In [211]:
########## TEST DATA #############
#transform into matrix given the dictionary
X_test = count_vect.transform(test_documents) 
#transform to array
X_test = X_test.toarray()
#the actual y values
y_test = []
for i in range(1,24): #authors: 1-23
    for j in range(0,5): #num of articles: 5 articles
        y_test.append(i)
#the prediction of y_test using MNB
predicted_test_y = mnb_clf.predict(X_test)
test_mnb_accuracy_rate = accuracy_score(y_test,predicted_test_y)
test_mnb_accuracy_rate

0.6695652173913044

In [212]:
print(precision_score(y_test, predicted_test_y, average = 'macro'))

0.70728088336784


/Users/duytienpham/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [213]:
print(recall_score(y_test, predicted_test_y, average = 'macro'))

0.6695652173913043


In [322]:
count_vect1 = CountVectorizer(stop_words = 'english',max_features = 10, ngram_range = (1,2))
# vect1 = count_vect1.fit(dictionary['author_1'])
# bow1 = vect1.transform(dictionary['author_1']).sum(axis = 0)
a1_str = ' '.join(dictionary_test['author_10'])
a1_lst = []
a1_lst.append(a1_str)
bow1 = count_vect1.fit(a1_lst)
bow1.vocabulary_
# bow1 = count_vect1.fit(a1_lst)
# bow2 = count_vect1.fit(dictionary['author_2'])
# bow3 = count_vect1.fit(dictionary['author_3'])
# bow4 = count_vect1.fit(dictionary['author_4'])
# bow5 = count_vect1.fit(dictionary['author_5'])
# bow6 = count_vect1.fit(dictionary['author_6'])
# bow9 = count_vect1.fit(dictionary['author_9'])
# bow12 = count_vect1.fit(dictionary['author_12'])
# bow13 = count_vect1.fit(dictionary['author_13'])
# bow14 = count_vect1.fit(dictionary['author_14'])


{'mom': 5,
 'got': 3,
 'soccer': 7,
 'tourney': 9,
 'did': 1,
 'school': 6,
 'game': 2,
 'keely': 4,
 'cities': 0,
 'summer': 8}

In [244]:
a1_str

"jenny kissed me when we met , jumping from the chair she sat in ; time , you thief , who love to get sweets into your list , put that in ! say i 'm weary , say i 'm sad , say that health and wealth have missed me , say i 'm growing old , but add , jenny kissed me . ~leigh hunt~  that 's where i went today with my mom , one of the best days i 've had since coming back . i love the city . i love that city . so , we headed down ( compliments of my driving , naturally ) and after visiting my great grandma at her `` older-people 's home '' ( which was so nice , she 's such a sweetly forgetful queen ) we headed up to the heart of toronto , via queen st. east . then drove by the university i want to go to next year ... if i 'm accepted . and around that area , checking out possible houses and stuff , since i wo n't be living in residence ( that university has none ) and i wo n't be living at home ( commuting is not an option ) . i was so excited to see where i 'll hopefully be in a year . i 

In [239]:
bow1.get_feature_names()

['bunny',
 'just',
 'like',
 'onions',
 'really',
 'round',
 'short',
 'subaru',
 've',
 'way']

In [269]:
bow2.get_feature_names()

['dont',
 'good',
 'im',
 'just',
 'like',
 'people',
 'really',
 'things',
 'time',
 'want']

In [36]:
bow3.get_feature_names()

['credits',
 'excited',
 'japanese',
 'just',
 'll',
 'people',
 'summer',
 'time',
 'want',
 'year']

# 2.2 TF-IDF Features

In [475]:
#tfidf: SVM linear
tfidf_vect = TfidfVectorizer(stop_words = 'english',max_features = 5000, ngram_range = (1,2))
tfidf_vectorizer = tfidf_vect.fit_transform(documents)
X_tfidf = tfidf_vectorizer.toarray()

svm_linear_clf = SVC(kernel = 'linear', decision_function_shape = 'ovo')
svm_linear_clf.fit(X_tfidf,y)
svm_linear_accuracy = cross_val_score(svm_linear_clf, X_tfidf,y, cv = 10)
np.mean(svm_linear_accuracy)

0.7456521739130435

In [476]:
#tfidf: SVM rbf
#tfidf: SVM linear
tfidf_vect = TfidfVectorizer(stop_words = 'english',max_features = 5000, ngram_range = (1,2))
tfidf_vectorizer = tfidf_vect.fit_transform(documents)
X_tfidf = tfidf_vectorizer.toarray()
svm_rbf_clf = SVC(kernel = 'rbf', decision_function_shape = 'ovo')
svm_rbf_clf.fit(X_tfidf,y)
svm_rbf_accuracy = cross_val_score(svm_rbf_clf, X_tfidf,y, cv = 10)
np.mean(svm_rbf_accuracy)

0.6652173913043479

In [477]:
#tfidf: Cosine Similarity
#applied L2
# X_normalized = normalize(X_tfidf, norm='l2', axis=1,return_norm=False)
tfidf_vect = TfidfVectorizer(stop_words = 'english',max_features = 5000, ngram_range = (1,2))
tfidf_vectorizer = tfidf_vect.fit_transform(documents)
X_tfidf = tfidf_vectorizer.toarray()
def cosine_similary_kernel(X,y):
    return cosine_similarity(X, y)
svm_cs_clf = SVC(kernel = 'linear', decision_function_shape = 'ovo')
svm_cs_clf.fit(X_tfidf ,y)
svm_cs_accuracy = cross_val_score(svm_cs_clf, X_tfidf ,y, cv = 10)
np.mean(svm_cs_accuracy)

0.7456521739130435

In [478]:
#Gaussian Naive Bayes
#tfidf: SVM linear
tfidf_vect = TfidfVectorizer(stop_words = 'english',max_features = 5000, ngram_range = (1,2))
tfidf_vectorizer = tfidf_vect.fit_transform(documents)
X_tfidf = tfidf_vectorizer.toarray()
gnb_clf = GaussianNB()
gnb_clf.fit(X_tfidf,y)
gnb_accuracy = cross_val_score(gnb_clf, X_tfidf,y, cv = 10)
np.mean(gnb_accuracy)

0.5956521739130436

In [479]:
#multinomial Log Reg
#tfidf: SVM linear
tfidf_vect = TfidfVectorizer(stop_words = 'english',max_features = 5000, ngram_range = (1,2))
tfidf_vectorizer = tfidf_vect.fit_transform(documents)
X_tfidf = tfidf_vectorizer.toarray()
mlr_clf = LogisticRegression(solver='lbfgs', multi_class='multinomial')
mlr_clf.fit(X_tfidf,y)
mlr_accuracy = cross_val_score(mlr_clf, X_tfidf,y, cv = 10)
np.mean(mlr_accuracy)

0.7869565217391303

In [480]:
tfidf_vect = TfidfVectorizer(stop_words = 'english',max_features = 5000, ngram_range = (1,2))
tfidf_vectorizer = tfidf_vect.fit_transform(documents)
test_X_tfidf = tfidf_vect.transform(test_documents)
test_X_tfidf = test_X_tfidf.toarray()
predicted_test_y = mlr_clf.predict(test_X_tfidf)
accuracy_rate = accuracy_score(y_test,predicted_test_y)
accuracy_rate

0.7652173913043478

In [481]:
print(precision_score(y_test, predicted_test_y, average = 'macro'))

0.8033643892339544


In [482]:
print(recall_score(y_test, predicted_test_y, average = 'macro'))

0.7652173913043478


In [1]:
# documents

In [109]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

new_doc = []
stop_words = set(stopwords.words('english'))

# for i in documents:
#     token = word_tokenize(i)
#     new_doc.append(token)


In [2]:
a1_str = ' '.join(dictionary['author_1'])
text = []
text.append(a1_str)
# text

In [264]:
a1_str = ' '.join(dictionary['author_1'])
text = []
text.append(a1_str)
# bow1 = count_vect1.fit(a1_lst)

In [277]:
tfidf_vect = TfidfVectorizer(max_features = 20000,stop_words = 'english',ngram_range = (1,2))
tfidf_vectorizer = tfidf_vect.fit_transform(text)
X_tfidf = tfidf_vectorizer.toarray()
X_names = tfidf_vect.get_feature_names()

In [414]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 5

# Run NMF
#nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=10, learning_method='online', learning_offset=50.,random_state=0).fit(X_tfidf)


/Users/duytienpham/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [415]:
df2 = pd.DataFrame(lda.transform(X_tfidf))
print(np.mean(df2,axis=0))

0    0.030454
1    0.038255
2    0.051576
3    0.046294
4    0.833421
dtype: float64


# Display top topics: not successful in this notebook - Refer to LDA notebook

In [1]:
# def display_topics(model, feature_names, no_top_words):
#     for topic_idx, topic in enumerate(model.components_):
#         print ("Topic %d:" % (topic_idx))
#         print (" ".join([feature_names[i]
#                         for i in topic.argsort()[:-no_top_words - 1:-1]]))

# no_top_words = 10
# #display_topics(nmf, tfidf_feature_names, no_top_words)
# display_topics(lda, X_names, no_top_words)

In [90]:
from nltk.corpus import wordnet
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [176]:
def lemmatizer(text):
    lemmatizer = WordNetLemmatizer()
    nltk_tagged= nltk.pos_tag(nltk.word_tokenize(text))
    wordnet_tagged = map(lambda x: (x[0], get_wordnet_pos(x[1])), nltk_tagged)
    lemmatized_lst = []
    for word, tag in wordnet_tagged:
#         if (word in stop_words == False):
        if tag is None:
            lemmatized_lst.append(word)
        else:
            lemmatized_lst.append(lemmatizer.lemmatize(word,tag))
#         else:
#             break
    return lemmatized_lst

In [181]:
import gensim
from gensim.models import ldamodel

In [182]:
corpus = []
for i in documents:
    token = lemmatizer(i)
    corpus.append(token)

In [180]:
for i in corpus:
    for j in range(len(i)):
        if ([i][j] in stop_words == True):
            corpus[i].remove(index(corpus[i][j]))
        else:
            continue

TypeError: unhashable type: 'list'

In [95]:
#flatten for a dictionary list
flattened_corpus = [item for sublist in corpus for item in sublist]

In [183]:
#remove stop words from the corpus
corpus

[['have',
  'an',
  'old',
  'clamshell',
  'ibook',
  'that',
  'i',
  'find',
  'on',
  'consignment',
  'work',
  'pretty',
  'well',
  'it',
  'be',
  'cheap',
  'than',
  'what',
  'be',
  'be',
  'sell',
  'on',
  'ebay',
  'and',
  'those',
  'be',
  'slow',
  'model',
  'add',
  'more',
  'memory',
  'so',
  'could',
  'upgrade',
  'os',
  'now',
  'run',
  'panther',
  'decently',
  'add',
  'a',
  'bunch',
  'of',
  'game',
  'that',
  'i',
  'just',
  'trash',
  'tonight',
  'to',
  'make',
  'more',
  'room',
  'on',
  'the',
  'hard',
  'drive',
  'only',
  'have',
  '10',
  'gig',
  'also',
  'be',
  'try',
  'to',
  'find',
  'replacement',
  'product',
  'for',
  'the',
  'microsoft',
  'product',
  'on',
  'it',
  'take',
  'off',
  'office',
  'and',
  'installed',
  'openoffice',
  'use',
  'netscape',
  'and',
  'safari',
  'instead',
  'of',
  'ie',
  'not',
  'exactly',
  'su',
  're',
  'why',
  'i',
  'm',
  'do',
  'it',
  'though',
  'it',
  'risk',
  'screw',

In [288]:
from gensim import corpora
id2word = corpora.Dictionary(corpus)
texts = corpus
corpus2 = [id2word.doc2bow(text) for text in texts]

In [185]:
# lst = [[1,2],[3,4]]
# b = [4,2]
# list(filter(lambda x:x[0] not in b,lst))

[[1, 2], [3, 4]]

In [187]:
# for i in range(len(lst)):
#     if j in range(len(lst[i])):
#         print([i][j])

[1, 2]
[3, 4]


In [254]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus2,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [255]:
print(lda_model.print_topics())
# doc_lda = lda_model[corpus]


[(0, '0.012*"the" + 0.007*"2" + 0.006*"child" + 0.006*"run" + 0.006*"n" + 0.004*"group" + 0.004*"of" + 0.004*"in" + 0.004*"will" + 0.003*"meeting"'), (1, '0.053*"i" + 0.039*"be" + 0.030*"the" + 0.028*"to" + 0.028*"and" + 0.020*"a" + 0.017*"it" + 0.015*"that" + 0.014*"of" + 0.014*"my"'), (2, '0.016*"ya" + 0.014*"vince" + 0.011*"danique" + 0.011*"dont" + 0.008*"x" + 0.007*"doesnt" + 0.006*"didnt" + 0.005*"email" + 0.004*"quixtar" + 0.004*"cant"')]
